In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime

In [2]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [3]:
def get_quandl_data(quandl_id):
    '''Pull and cache Quandl data into a dataframe. Caching is important for avoiding rate limits.'''
    cache_path = '{}.pkl'.format(quandl_id).replace('/','-')
    try:
        # Check if the pickle file is in the working directory.
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        # The data is not locally avaiable. Doenload it and store it in a form of pickle file. 
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id,start_date='2019-03-11', end_date='2020-03-11', returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df


In [4]:

btc_eur_price_kraken = get_quandl_data('BCHARTS/BtcdeEUR')


Loaded BCHARTS/BtcdeEUR from cache


In [5]:
btc_eur_price_kraken.head()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2019-03-11,3500.00,6665.93,3300.33,3403.32,167.596995,576396.316320,3439.180490
2019-03-12,3460.65,5999.41,3201.00,3477.15,151.268027,521429.524254,3447.057074
2019-03-13,3410.00,29985.01,3317.71,3417.99,134.693142,465207.072297,3453.828931
2019-03-14,3442.89,29985.01,3204.00,3478.84,173.722470,593900.178186,3418.672199
2019-03-15,3390.85,7903.37,3200.00,3463.59,169.784075,583656.846927,3437.641890


In [6]:
# Chart the BTC pricing data
btc_trace = go.Scatter(x=btc_eur_price_kraken.index, y=btc_eur_price_kraken['Weighted Price'])
py.iplot([btc_trace])

In [7]:
# Pull pricing data for 3 more BTC exchanges
exchanges = ['COINFALCON','BITSTAMP','CEX']

exchange_data = {}

exchange_data['KRAKEN'] = btc_eur_price_kraken

for exchange in exchanges:
    exchange_code = 'BCHARTS/{}EUR'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code)
    exchange_data[exchange] = btc_exchange_df

Loaded BCHARTS/COINFALCONEUR from cache
Loaded BCHARTS/BITSTAMPEUR from cache
Loaded BCHARTS/CEXEUR from cache


In [8]:
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)

In [9]:
# Merge the BTC price dataseries' into a single dataframe
btc_eur_datasets = merge_dfs_on_column(list(exchange_data.values()), list(exchange_data.keys()), 'Weighted Price')

In [10]:
btc_eur_datasets.head()

,KRAKEN,COINFALCON,BITSTAMP,CEX
Date,,,,
2019-03-11,3439.180490,3448.184516,3428.999223,3436.571582
2019-03-12,3447.057074,3433.578904,3426.772235,3426.213298
2019-03-13,3453.828931,3429.055697,3408.807869,3411.501663
2019-03-14,3418.672199,3400.578557,3405.760567,3412.842695
2019-03-15,3437.641890,3437.488020,3429.877559,3431.657184


In [11]:
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = True
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)

In [12]:
# Plot all of the BTC exchange prices
df_scatter(btc_eur_datasets, 'Bitcoin Price (EUR) By Exchange')

In [13]:
# Plot all of the BTC exchange prices
df_scatter(btc_eur_datasets, 'Bitcoin Price (EUR) By Exchange')

In [14]:
# Remove "0" values
btc_eur_datasets.replace(0, np.nan, inplace=True)

In [15]:
# Plot the revised dataframe
df_scatter(btc_eur_datasets, 'Bitcoin Price (EUR) By Exchange')

In [16]:
# Calculate the average BTC price as a new column
btc_eur_datasets['avg_btc_price_eur'] = btc_eur_datasets.mean(axis=1)

In [17]:
# Plot the average BTC price
btc_trace = go.Scatter(x=btc_eur_datasets.index, y=btc_eur_datasets['avg_btc_price_eur'])
py.iplot([btc_trace])

In [18]:
def get_json_data(json_url, cache_path):
    '''Download and cache JSON data, return as a dataframe.'''
    
    try:
        # Check if the pickle file is in the working directory.
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(json_url))
    except (OSError, IOError) as e:
        # The data is not locally avaiable. Doenload it and store it in a form of pickle file. 
        print('Downloading {}'.format(json_url))
        df = pd.read_json(json_url)
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(json_url, cache_path))
    return df

In [19]:
base_polo_url = 'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'
start_date = datetime.strptime('2019-03-11', '%Y-%m-%d')
end_date = datetime.strptime('2020-03-11', '%Y-%m-%d')
pediod = 86400 # pull daily data (86,400 seconds per day)

def get_crypto_data(poloniex_pair):
    '''Retrieve cryptocurrency data from poloniex'''
    json_url = base_polo_url.format(poloniex_pair, start_date.timestamp(), end_date.timestamp(), pediod)
    data_df = get_json_data(json_url, poloniex_pair)
    data_df = data_df.set_index('date')
    return data_df

In [20]:
altcoins = ['ETH','LTC','XRP','ETC','STR','DASH','SC','XMR','XEM']

altcoin_data = {}
for altcoin in altcoins:
    coinpair = 'BTC_{}'.format(altcoin)
    crypto_price_df = get_crypto_data(coinpair)
    altcoin_data[altcoin] = crypto_price_df

Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETH&start=1552258800.0&end=1583881200.0&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_LTC&start=1552258800.0&end=1583881200.0&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XRP&start=1552258800.0&end=1583881200.0&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETC&start=1552258800.0&end=1583881200.0&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_STR&start=1552258800.0&end=1583881200.0&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_DASH&start=1552258800.0&end=1583881200.0&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_SC&start=1552258800.0&end=1583881200.0&period=86400 from cache
Loaded https://polon

In [21]:
altcoin_data['ETH'].tail()

,high,low,open,close,volume,quoteVolume,weightedAverage
date,,,,,,,
2020-03-06,0.026841,0.025193,0.025208,0.026830,538.199715,20687.078368,0.026016
2020-03-07,0.027477,0.026584,0.026830,0.026698,405.429524,15064.944610,0.026912
2020-03-08,0.026684,0.024431,0.026684,0.024781,1521.246389,59274.747567,0.025664
2020-03-09,0.026168,0.024695,0.024812,0.025572,1414.432012,55581.905215,0.025448
2020-03-10,0.025747,0.025067,0.025574,0.025409,493.181011,19419.418707,0.025396


In [22]:
# Calculate EUR Price as a new column in each altcoin dataframe
for altcoin in altcoin_data.keys():
    altcoin_data[altcoin]['price_eur'] =  altcoin_data[altcoin]['weightedAverage'] * btc_eur_datasets['avg_btc_price_eur']

In [23]:
# Merge EUR price of each altcoin into single dataframe 
combined_df = merge_dfs_on_column(list(altcoin_data.values()), list(altcoin_data.keys()), 'price_eur')

In [24]:
# Add BTC price to the dataframe
combined_df['BTC'] = btc_eur_datasets['avg_btc_price_eur']

In [25]:
# Chart all of the altocoin prices
df_scatter(combined_df, 'Cryptocurrency Prices (EUR)', seperate_y_axis=False, y_axis_label='Coin Value (EUR)', scale='log')

In [26]:
combined_df.pct_change().corr(method='pearson')

,ETH,LTC,XRP,ETC,STR,DASH,SC,XMR,XEM,BTC
ETH,1.000000,0.799575,0.797606,0.658775,0.753952,0.673727,0.646857,0.778489,0.705649,0.771455
LTC,0.799575,1.000000,0.707934,0.636152,0.678562,0.611981,0.572497,0.717569,0.611671,0.705452
XRP,0.797606,0.707934,1.000000,0.602177,0.795103,0.590175,0.556498,0.710912,0.610051,0.637255
ETC,0.658775,0.636152,0.602177,1.000000,0.610057,0.613515,0.458257,0.612845,0.554388,0.545885
STR,0.753952,0.678562,0.795103,0.610057,1.000000,0.577084,0.561709,0.656474,0.678100,0.528115
DASH,0.673727,0.611981,0.590175,0.613515,0.577084,1.000000,0.489882,0.669534,0.501595,0.591262
SC,0.646857,0.572497,0.556498,0.458257,0.561709,0.489882,1.000000,0.572751,0.565721,0.546201
XMR,0.778489,0.717569,0.710912,0.612845,0.656474,0.669534,0.572751,1.000000,0.592814,0.742034
XEM,0.705649,0.611671,0.610051,0.554388,0.678100,0.501595,0.565721,0.592814,1.000000,0.551671
BTC,0.771455,0.705452,0.637255,0.545885,0.528115,0.591262,0.546201,0.742034,0.551671,1.000000


In [27]:
def correlation_heatmap(df, title, absolute_bounds=True):
    '''Plot a correlation heatmap for the entire dataframe'''
    heatmap = go.Heatmap(
        z=df.corr(method='pearson'),
        x=df.columns,
        y=df.columns,
        colorbar=dict(title='Pearson Coefficient'),
    )
    
    layout = go.Layout(title=title)
    
    if absolute_bounds:
        heatmap['zmax'] = 1.0
        heatmap['zmin'] = -1.0
        
    fig = go.Figure(data=[heatmap], layout=layout)
    py.iplot(fig)

In [28]:
correlation_heatmap(combined_df.pct_change(), "Cryptocurrency Correlations in 2019-2020")